# `method` on zf

# running `method` on zf

## imports

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
!pip install scanpy -q

In [23]:
!pip install scprep -q

In [24]:
!pip install moscot -q

In [25]:
!pip install "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html -q

In [26]:
import pandas as pd
import scanpy as sc
import numpy as np
import torch
import sys
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import importlib
import scprep
import anndata
import tqdm
import networkx as nx
import scipy
from scipy.spatial import distance
from sklearn.neighbors import kneighbors_graph
import moscot as mt
import moscot.plotting as mpl
from moscot.problems.time import TemporalProblem
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [27]:
filehandle_wdm = 'drive/Othercomputers/numac/GitHub/WDM/'
filehandle_save = 'drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_moscot/'
# filehandle_save_factors = 'drive/Othercomputers/numac/GitHub/celltypediscovery/_wdm_save_sc_factors/'
filehandle_zf = 'drive/MyDrive/DX/_data/zebrafish/cleaned_common_pca_sc/'

sys.path.insert(0, filehandle_wdm)
sys.path.insert(0, filehandle_save)
sys.path.insert(0, filehandle_zf)

import clustering
import util_LR
import util_zf
import FRLC_LRDist

## reload

In [28]:
importlib.reload(clustering)
importlib.reload(util_LR)
importlib.reload(util_zf)
importlib.reload(FRLC_LRDist)

<module 'FRLC_LRDist' from '/content/drive/Othercomputers/numac/GitHub/WDM/FRLC_LRDist.py'>

## device

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'On device: {device}')
dtype = torch.float64

On device: cuda


## load zf: spatial, annotation hard clusterings as $Q$'s

In [30]:
# daniocell_dir = '/scratch/gpfs/ph3641/zebrafish/daniocell.h5ad'
# adata_daniocell = sc.read_h5ad(daniocell_dir)
# stages = np.unique(adata_daniocell.obs['hpf'].values)

zf_list = ['zf3', 'zf5', 'zf10', 'zf12', 'zf18', 'zf24']
zf_names = zf_list
filehandles_zf = [filehandle_zf + 'pair' + str(k) + '/' for k in range(len(zf_list))]
N = len(filehandles_zf)

spatial_list = []
exclude_rows = [None, None, None, None, None, None] # [1099, None, None, 325, None, None]

for i in range(len(zf_list)):
    file_spatial = filehandles_zf[i] + zf_names[i] +'_umap.npy'
    if i == len(zf_list) - 1:
        file_spatial = filehandles_zf[i-1] + zf_names[i] +'_umap.npy'
    spatial = np.load(file_spatial)
    nidx = exclude_rows[i]
    if nidx is not None:
        spatial = np.concatenate((spatial[:nidx,:], spatial[nidx+1:,:]))
    spatial_list.append(spatial)

In [31]:
N = len(filehandles_zf)
Qs = [None]*(N-1)
Rs = [None]*(N-1)
adatas_Qs = [None] * (N-1)
adatas_Rs = [None] * (N-1)

key = 'celltype_new'

labels = []

for i in range(N-2, -1, -1):

    s1_name = zf_names[i]
    s2_name = zf_names[i+1]

    print(f'Slice pair {i}, aligning {s1_name} to {s2_name}')

    adata_pair = sc.read_h5ad(filehandle_zf + f'pair{i}/' + s1_name + '_' + s2_name + '.h5ad')

    adata1 = adata_pair[adata_pair.obs['timepoint'] == 1]
    adata2 = adata_pair[adata_pair.obs['timepoint'] == 2]
    adatas_Qs[i], adatas_Rs[i] = adata1, adata2

    encoder1 = OneHotEncoder(sparse_output=False)
    ys_onehot1 = encoder1.fit_transform(adata1.obs[key].values.reshape(-1, 1))
    _Q = ys_onehot1 / np.sum(ys_onehot1)

    encoder2 = OneHotEncoder(sparse_output=False)
    ys_onehot2 = encoder2.fit_transform(adata2.obs[key].values.reshape(-1, 1))
    _R = ys_onehot2 / np.sum(ys_onehot2)

    labels.append(list(encoder2.categories_[0]))

    if i == 0:
        labels.append(list(encoder1.categories_[0]))

    _Q, _R = torch.from_numpy(_Q).to(device).float(), torch.from_numpy(_R).to(device).float()

    # Filter rows
    nidx_1, nidx_2 = exclude_rows[i], exclude_rows[i+1]
    if nidx_1 is not None:
        _Q = torch.cat((_Q[:nidx_1,:], _Q[nidx_1+1:,:]))
    if nidx_2 is not None:
        _R = torch.cat((_R[:nidx_2,:], _R[nidx_2+1:,:]))

    Qs[i], Rs[i] = _Q, _R

Qs_gt_tensor = Qs + [Rs[4]]
Qs_gt_tensor = [ Q.type(torch.DoubleTensor).to(device) for Q in Qs_gt_tensor]

Qs_gt = [Q.cpu().numpy() for Q in Qs_gt_tensor]

adatas = adatas_Qs + [adatas_Rs[-1]]

# Extract cell type labels directly from adatas
ct_labels = [list(adata.obs[key].unique()) for adata in adatas]

Slice pair 4, aligning zf18 to zf24
Slice pair 3, aligning zf12 to zf18
Slice pair 2, aligning zf10 to zf12
Slice pair 1, aligning zf5 to zf10
Slice pair 0, aligning zf3 to zf5


In [32]:
for la in ct_labels:
    print(len(la))

2
9
8
12
14
23


## set ranks

In [33]:
print([Q.shape for Q in Qs_gt])

[(7424, 2), (7136, 9), (8755, 8), (12133, 12), (9903, 14), (14882, 23)]


In [34]:
ranks = [(2,9),(9,8),(8,12),(12,14),(14, 23)]

## load features and spatial coords specific to the triple

In [35]:
s0_name = zf_names[0]
s1_name = zf_names[1]
s2_name = zf_names[2]
s3_name = zf_names[3]
s4_name = zf_names[4]
s5_name = zf_names[5]


filehandle_pair0 = filehandle_zf + 'pair' + str(0) + '/'
filehandle_pair1 = filehandle_zf + 'pair' + str(1) + '/'
filehandle_pair2 = filehandle_zf + 'pair' + str(2) + '/'
filehandle_pair3 = filehandle_zf + 'pair' + str(3) + '/'
filehandle_pair4 = filehandle_zf + 'pair' + str(4) + '/'


X0 = np.load(filehandle_pair0 + s0_name + '_feature.npy')
X1 = np.load(filehandle_pair1 + s1_name + '_feature.npy')
X2 = np.load(filehandle_pair2 + s2_name + '_feature.npy')
X3 = np.load(filehandle_pair3 + s3_name + '_feature.npy')
X4 = np.load(filehandle_pair4 + s4_name + '_feature.npy')
X5 = np.load(filehandle_pair4 + s5_name + '_feature.npy')

Xs = [X0, X1, X2, X3, X4, X5]
Ss = spatial_list

## set OT hyperparameters

In [36]:
tau_a = .999999
tau_b = .999999

# max_iter=100
# min_iter=100

new_expr_signal=False

use_bin_annotation=True
n_top_genes='all'
n_comps_pca=30

print(f"""
Data Processing and moscot Parameters:
-----------------------
Tau_a (Out): {tau_a}
Tau_b (In): {tau_b}
New Expression Signal: {new_expr_signal}
Using Cell Type Anno: {use_bin_annotation}
Number of Top Genes: {n_top_genes} (if Anno=True, ~5k after DE from Anno)
Number of PCA components: {n_comps_pca}
""")


Data Processing and moscot Parameters:
-----------------------
Tau_a (Out): 0.999999
Tau_b (In): 0.999999
New Expression Signal: False
Using Cell Type Anno: True
Number of Top Genes: all (if Anno=True, ~5k after DE from Anno)
Number of PCA components: 30



In [37]:
# Compute alignment
for i in tqdm.tqdm(range(N-2)):

    s1_name = zf_names[i]
    s2_name = zf_names[i+1]

    print(f'Slice pair {i}, aligning {s1_name} to {s2_name}')

    pair = sc.read_h5ad(filehandle_zf + f'pair{i}/' + s1_name + '_' + s2_name + '.h5ad')

    print(pair)


  0%|          | 0/4 [00:00<?, ?it/s]

Slice pair 0, aligning zf3 to zf5


 25%|██▌       | 1/4 [00:00<00:01,  1.69it/s]

AnnData object with n_obs × n_vars = 14560 × 7000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'celltype_new', 'time', 'n_genes', 'timepoint'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'
Slice pair 1, aligning zf5 to zf10


 50%|█████     | 2/4 [00:01<00:01,  1.81it/s]

AnnData object with n_obs × n_vars = 15891 × 7000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'celltype_new', 'time', 'n_genes', 'timepoint'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'
Slice pair 2, aligning zf10 to zf12


 75%|███████▌  | 3/4 [00:01<00:00,  1.62it/s]

AnnData object with n_obs × n_vars = 20888 × 7000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'celltype_new', 'time', 'n_genes', 'timepoint'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'
Slice pair 3, aligning zf12 to zf18


100%|██████████| 4/4 [00:02<00:00,  1.56it/s]

AnnData object with n_obs × n_vars = 22036 × 7000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'celltype_new', 'time', 'n_genes', 'timepoint'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts'


In [38]:
%%time
# Compute alignment
for i in tqdm.tqdm(range(N-1)):

    s1_name = zf_names[i]
    s2_name = zf_names[i+1]

    print(f'Slice pair {i}, aligning {s1_name} to {s2_name}')

    pair = sc.read_h5ad(filehandle_zf + f'pair{i}/' + s1_name + '_' + s2_name + '.h5ad')

    timepoints = pair.obs['timepoint']
    marginal = np.zeros((timepoints == 1).sum())

    pair.obs['timepoint'] = pair.obs['timepoint'].astype('category')
    pair.obsm['feature'] = pair.X

    tp = TemporalProblem(pair)
    tp = tp.prepare(time_key='timepoint', joint_attr='X_pca')
    tp = tp.solve(epsilon=1e-3, tau_a=tau_a, tau_b=tau_b, scale_cost="mean")

    pi = tp.solutions[(1,2)].transport_matrix
    pi = np.array(pi)

    np.save(filehandle_save + f'moscot_Pi_{i}.npy', pi)  ### SAVE

  0%|          | 0/5 [00:00<?, ?it/s]

Slice pair 0, aligning zf3 to zf5
INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(7424, 7136)].                                  


 20%|██        | 1/5 [00:08<00:34,  8.57s/it]

Slice pair 1, aligning zf5 to zf10
INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(7136, 8755)].                                  


 40%|████      | 2/5 [00:23<00:37, 12.54s/it]

...

Slice pair 3, aligning zf12 to zf18
INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(12133, 9903)].                                 
WARNING  Solver did not converge                                                                                   


 80%|████████  | 4/5 [01:55<00:35, 35.14s/it]

Slice pair 4, aligning zf18 to zf24
INFO     Solving `1` problems                                                                                      
INFO     Solving problem BirthDeathProblem[stage='prepared', shape=(9903, 14882)].                                 
WARNING  Solver did not converge                                                                                   


100%|██████████| 5/5 [02:55<00:00, 35.20s/it]

CPU times: user 2min 47s, sys: 4.25 s, total: 2min 51s
Wall time: 2min 55s


In [39]:
# to get transition matrix
# T = ((Q1.T @ P) @ Q2)*(Q1.shape[0]*Q2.shape[0])

'''
for i in range(N-1):
    P = np.save(filehandle_save + f'moscot_Pi_{i}.npy', pi)  ### SAVE

print(f'gQ1: {gQ1}')
    print(f'gQ2: {gQ2}')
    print(f'T sum 1: {np.sum(T, axis=0)}, sum 2: {np.sum(T, axis=1)}')
'''

"\nfor i in range(N-1):\n    P = np.save(filehandle_save + f'moscot_Pi_{i}.npy', pi)  ### SAVE\n\nprint(f'gQ1: {gQ1}')\n    print(f'gQ2: {gQ2}')\n    print(f'T sum 1: {np.sum(T, axis=0)}, sum 2: {np.sum(T, axis=1)}')\n"

## plot

In [40]:
Ss = spatial_list

node_labels1 = [None for Q in WDM.Q_gammas]

clustering.plot_clusters_from_QT(Ss, Qs_pred, Ts_pred, node_labels1, dotsize=10)
clustering.diffmap_from_QT(Qs_pred, Ts_pred, cell_type_labels=node_labels1, dsf=.3)

NameError: name 'WDM' is not defined

## save

In [ ]:
for i,Q in enumerate(WDM.Q_gammas):
    np.save(filehandle_save + f'wdm_Q{i}.npy', Q.cpu().numpy())

for i,T in enumerate(WDM.T_gammas):
    np.save(filehandle_save + f'wdm_T{i}{i+1}.npy', T.cpu().numpy())

## ari, ami

In [ ]:
importlib.reload(util_zf)

In [ ]:
gt_types_list = clustering.max_likelihood_clustering(Qs_gt)
pred_types_list = clustering.max_likelihood_clustering(Qs_pred)

util_zf.compute_ARI_and_AMI(gt_types_list, pred_types_list)

In [ ]:
importlib.reload(util_zf)

In [ ]:
# util_zf.silhouette(gt_types_list, pred_types_list, Xs, Ss)

In [ ]:
# util_zf.cos_silhouette(gt_types_list, pred_types_list, Xs, Ss)

## freezing $Q$s with annotations, running `WDM`

In [ ]:
%%time
Q_annot = Qs_gt_tensor

importlib.reload(WassersteinDiffMap)

WDM2 = WassersteinDiffMap.WassersteinDifferentiationMapping(ranks, tau_in = tau_in, tau_out=tau_out, \
                      gamma=gamma, max_iter=100, min_iter=100, device=device, dtype=torch.float64, \
                     printCost=True, returnFull=False, alpha=alpha, beta=beta, \
                      initialization='Full', init_args=None)

WDM2.impute_annotated_transitions(C_factors_sequence, A_factors_sequence, Q_annot)
T_gammas = WDM2.T_gammas

In [ ]:
for i,Q in enumerate(WDM2.Q_gammas):
    np.save(filehandle_save + f'ann_Q{i}.npy', Q.cpu().numpy())

for i,T in enumerate(WDM2.T_gammas):
    np.save(filehandle_save + f'ann_T{i}{i+1}.npy', T.cpu().numpy())

# Analysis part 1: entropy of transitions

In [ ]:
Ts_ann = T_gammas
Ts_pred = Ts_pred

## entropy functions

In [ ]:
importlib.reload(util_zf)

In [ ]:
util_zf.compare_T_entropies(Ts_ann, Ts_pred)
print('\n')
util_zf.compare_T_col_entropies(Ts_ann, Ts_pred)

## convergence / collision

In [ ]:
importlib.reload(util_zf)

In [ ]:
for i, (T_ann, T_pred) in enumerate(zip(Ts_ann, Ts_pred)):
    T_ann = T_ann.cpu().numpy()
    util_zf.plot_collision_profiles(T_ann, T_pred, title=f'Convergence profiles at slice {i+2}')

## anno diffmap

In [ ]:
clustering.plot_clusters_from_QT(Ss, Qs_gt, Ts_ann, ct_labels, dotsize=10)
clustering.diffmap_from_QT(Qs_gt, Ts_ann, cell_type_labels=ct_labels, dsf=.3)

## what about arg-secondmax? Does this have biological signal?

In [ ]:
labels_2nd = []

for Q in Qs_pred:
    # Get the indices of the sorted entries for each row
    sorted_indices = np.argsort(Q, axis=1)

    # Extract the second-to-last index (the second largest) from the sorted indices
    second_largest_indices = sorted_indices[:, -2]
    labels_2nd += [second_largest_indices]

In [ ]:
threshold = 0.1  # Example threshold as a percentage of the largest value

labels_consensus = []

for Q in Qs_pred:
    argmax_labels = np.argmax(Q, axis=1)
    sorted_indices = np.argsort(Q, axis=1)
    second_argmax_labels = sorted_indices[:, -2]

    # Get the largest and second largest values from Q
    largest_values = np.max(Q, axis=1)
    second_largest_values = Q[np.arange(Q.shape[0]), second_argmax_labels]

    # Apply the threshold to choose between argmax and second argmax
    consensus_labels = np.where((largest_values - second_largest_values) / largest_values < threshold,
                                second_argmax_labels, argmax_labels)

    # Convert consensus_labels (NumPy array) to a regular Python list of ints
    labels = consensus_labels.tolist()  # Convert the array to a list
    labels = [int(label) for label in labels]  # Ensure all elements are ints
    labels = np.array(labels)
    labels_consensus.append(labels)  # Use append to add the new list of labels

# Now labels_consensus is a list of lists, suitable for plotting
clustering.plot_clustering_list(Ss, labels_consensus, dotsize=10)

In [ ]:
util_zf.compute_ARI_and_AMI(gt_types_list, labels_consensus)